In [1]:
import numpy as np
import pandas as pd
import os
import csv
import re
from dotenv import load_dotenv
import json
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import glob
import PyPDF2 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from PyPDF2 import PdfReader
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import hashlib
from difflib import SequenceMatcher
import time

/home/cptaswadu/new-rescue/RESCUE-n8n/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = '/home/cptaswadu/new-rescue/RESCUE-n8n'
load_dotenv(dotenv_path=os.path.join(path, ".env"))
openai_api_key = os.getenv("OPEN_AI_API_KEY")

In [3]:
answer_policy_dir = "/home/cptaswadu/new-rescue/RESCUE-n8n/eval/insurance/dataset/policy_answer_real"
for filename in os.listdir(answer_policy_dir):
    file_path = os.path.join(answer_policy_dir, filename)
    if os.path.isfile(file_path):
        with open(file_path, "rb") as f:
            content = f.read()
            md5_hash = hashlib.md5(content).hexdigest()
        print(f"{filename} => {md5_hash}")

BCBS_FEP_20402 Germline Genetic Testing for.pdf => c5c2b854957d06467835e88a963d0c82
Cigna_MOL.TS_.150A_CMA_for_Developmental_Disorders_and_Prenatal_Diagnosis_eff01.01.2025_pub09.10.2024_1.pdf => dd74cd39fca15b7b0888b16ce1da2014
Cigna_MOL.TS_.344.A_Chromosomal_Microarray_Solid_Tumors_eff01.01.2025_pub09.11.2024_0.pdf => 49b51c0399e5563339f32a9f24f20641
BCBS_FEP_20459 Genetic Testing for Developmental.pdf => 8340e5b0ce4959eccfb2cb295edb47f3
Cigna_MOL.TS_.306.C_Whole_Genome_Sequencing_Cigna_eff01.01.2025_pub09.10.2024.pdf => 36bb5264dda1b2027dcdfdd32a714204
BCBS_FEP_204102 Whole Exome and.pdf => d5e9701c13de1dca302ad0ce45524039
Cigna_MOL.TS_.238.A_BRCA_Analysis_eff01.01.2025_pub09.10.2024_1.pdf => 626eac4d60df057ea93ece78f8cc3dfc
United Healthcare_genetic-testing-hereditary-cancer.pdf => c69485372670ce1d12aa8f61d83a06fd
United Healthcare_whole-exome-and-whole-genome-sequencing.pdf => 4fadf6b3ca9d4d08131cb31365e3aa7d
Cigna_MOL.TS_.235.C_Whole_Exome_Sequencing_Cigna_eff01.01.2025_pub09.20.2

In [ ]:
class QnAExecutorWithPDF:
    uploaded_files_cache = {} # Class-level cache: MD5 → uploaded OpenAI file_id
    def __init__(self, questions_list, llm_model="chatgpt", openai_api_key=None):
        # Initialize with questions, model name, and API key
        self.questions_list = questions_list
        self.formatted_questions = self.format_questions()
        self.llm_model = llm_model
        self.openai_api_key = openai_api_key
        self.assistant_id = None
        if openai_api_key:
            self.openai_client = OpenAI(api_key=openai_api_key)
        else:
            self.openai_client = None
        if self.openai_client:
            self.assistant_id = self._create_reusable_assistant()

        self.pdf_base_path = "/home/cptaswadu/new-rescue/RESCUE-n8n/eval/insurance/dataset/policy_answer_real"

        self.md5_to_pdf = {
            "c5c2b854957d06467835e88a963d0c82": "BCBS_FEP_20402 Germline Genetic Testing for.pdf",
            "8340e5b0ce4959eccfb2cb295edb47f3": "BCBS_FEP_20459 Genetic Testing for Developmental.pdf",
            "d5e9701c13de1dca302ad0ce45524039": "BCBS_FEP_204102 Whole Exome and.pdf",
            "36bb5264dda1b2027dcdfdd32a714204": "Cigna_MOL.TS_.306.C_Whole_Genome_Sequencing_Cigna_eff01.01.2025_pub09.10.2024.pdf",
            "626eac4d60df057ea93ece78f8cc3dfc": "Cigna_MOL.TS_.238.A_BRCA_Analysis_eff01.01.2025_pub09.10.2024_1.pdf",
            "c69485372670ce1d12aa8f61d83a06fd": "United Healthcare_genetic-testing-hereditary-cancer.pdf",
            "8a7d5f974648c666b635eae9e03277e7": "United Healthcare_chromosome-microarray-testing.pdf",
            "dd74cd39fca15b7b0888b16ce1da2014": "Cigna_MOL.TS_.150A_CMA_for_Developmental_Disorders_and_Prenatal_Diagnosis_eff01.01.2025_pub09.10.2024_1.pdf",
            "4fadf6b3ca9d4d08131cb31365e3aa7d": "United Healthcare_whole-exome-and-whole-genome-sequencing.pdf",
            "ad2eb3a750b767e32ff847032f0e8e03": "Cigna_MOL.TS_.235.C_Whole_Exome_Sequencing_Cigna_eff01.01.2025_pub09.20.2024.pdf",
            "49b51c0399e5563339f32a9f24f20641": "Cigna_MOL.TS_.344.A_Chromosomal_Microarray_Solid_Tumors_eff01.01.2025_pub09.11.2024_0.pdf"
        }

    def _create_reusable_assistant(self):
        """Create assistant once and reuse for all cases"""
        json_example = """{
  "Q0": "WES",
  "Q1": "Yes",
  "Q2": "Not Specified",
  "Q3": "Not Specified",
  "Q4": "No",
  "Q5": "No", 
  "Q6": "Not Specified",
  "Q7": "81415",
  "Q8": "No"
}"""
        assistant = self.openai_client.beta.assistants.create(
            name="Insurance Policy Analyzer",
            instructions=f"""
You are a clinical insurance assistant specializing in genetic testing coverage policies.
You MUST answer in JSON format only.

You will be given:

1. Patient clinical information for answering questions.
2. Official insurance policy document text (strictly use this policy content for answering questions).

Instructions:
- Answer all questions strictly based on the insurance policy document provided.
- Do NOT refer to general guidelines or policies from other insurance providers.
- If policy document does not clearly specify rules, you MAY use patient's clinical information to infer answers carefully.
- Do NOT assume coverage criteria from other insurers or general clinical guidelines unless explicitly stated in the policy.
- Output answers in JSON format ONLY.

Focus on sections for uploaded policy document:
- **Age criteria**
- **Medical necessity criteria**
- **Prior test criteria**
- **Family history information** 
- **Related CPT codes**
- **Coverage criteria**
- **Counseling / Provider criteria**

Based on the uploaded policy document and patient information, answer these questions:
{self.formatted_questions}

Output your answers in JSON format only, with no explanation.
Your response must follow this exact structure:
{json_example}

Answer options for each question:
- Q0: ["WES", "WGS", "BRCA1/2", "CMA"]
- Q1: ["Yes", "No", "Not Specified"]
- Q2: ["Yes", "No", "Not Specified"]
- Q3: ["Yes", "No", "Not Specified"]
- Q4: ["Yes", "No", "Not Specified"]
- Q5: ["Yes", "No", "Not Specified"]
- Q6: ["Yes", "No", "Not Specified"]
- Q7: ["81162", "81277", "81228", "81415", "81425", "Not Specified"]
- Q8: ["Yes", "No"]
""",
            model="gpt-4o",
            tools=[{"type": "file_search"}]
        )
        return assistant.id

    def get_pdf_path_from_case(self, case_data):
        '''Resolve the expected PDF path from case's MD5'''
        if 'expected_md5' not in case_data:
            raise ValueError(f"Case {case_data.get('id', 'Unknown')} does not have 'expected_md5' field")

        md5_hash = case_data['expected_md5']

        # Just in case md5 Unknown cases
        if md5_hash == "UNKNOWN":
            print(f"⚠️ {case_data.get('id', 'Unknown')}: UNKNOWN MD5 - proceeding without PDF")
            return None, md5_hash

        if md5_hash not in self.md5_to_pdf:
            print(f"⚠️ MD5 hash {md5_hash} not found in mapping - proceeding without PDF")
            return None, md5_hash

        pdf_filename = self.md5_to_pdf[md5_hash]
        pdf_path = os.path.join(self.pdf_base_path, pdf_filename)
        if not os.path.exists(pdf_path):
            print(f"⚠️ PDF file does not exist: {pdf_path} - proceeding without PDF")
            return None, md5_hash

        print(f"✅ {case_data['id']}: Found PDF - {pdf_filename} - {md5_hash}")
        return pdf_path, md5_hash

    def upload_pdf_to_openai(self, pdf_path, md5_hash):
        '''Upload PDF to OpenAI; reuse cached file if available'''
        if md5_hash in self.uploaded_files_cache:
            cached_file_id = self.uploaded_files_cache[md5_hash]
            print(f"✅ Utilizing cached file: {cached_file_id} (MD5: {md5_hash})")
            return cached_file_id
        
        try:
            with open(pdf_path, "rb") as file:
                uploaded_file = self.openai_client.files.create(
                    file=file,
                    purpose="assistants"
                )
            self.uploaded_files_cache[md5_hash] = uploaded_file.id
            print(f"✅ OpenAI upload successed: {uploaded_file.id} (MD5: {md5_hash})")
            return uploaded_file.id
        except Exception as e:
            print(f"❗ OpenAI upload failed: {e}")
            raise e

    def format_question_block(self, q, indent=2):
        '''Format a single question with its options'''
        indent_str = " " * indent
        question_line = f"{q['question']}"
        question_line += f"\n{indent_str}Options: {', '.join(q['options'])}"
        return question_line

    def format_questions(self):
        '''Join all formatted questions with IDs and spacing'''
        return "\n\n".join([
            f"{q['id']}. {self.format_question_block(q)}"
            for q in self.questions_list
        ])

    def clean_json_response(self, response_text):
        """Return a parsed JSON object from an LLM response, stripping code fences and extra text."""
    # Clean and extract JSON from the response text
        original = response_text.strip()

    # Step 0: Check for hallucinated greeting (Perplexity fallback)
        if "how can I assist you" in original.lower() or "insurance-related questions" in original.lower():
            raise ValueError("Perplexity returned generic assistant response instead of JSON.")

    # Step 1: Try direct parsing
        try:
            return json.loads(original)
        except json.JSONDecodeError:
            pass

    # Step 2: Remove code block wrappers
        cleaned = re.sub(r"^```(?:json)?\s*|\s*```$", "", original, flags=re.IGNORECASE).strip()
        try:
            return json.loads(cleaned)
        except json.JSONDecodeError:
            pass

    # Step 3: Try to extract the first {...} JSON-like block
        match = re.search(r"(\{[\s\S]*?\})", original)
        if match:
            try:
                return json.loads(match.group(1))
            except json.JSONDecodeError:
                pass

        raise ValueError("No valid JSON found in the response.")
    
    def run_qna_with_pdf(self, case_data, qna_model="gpt4o"):
        """run QnA with PDF document using OpenAI API."""
        case_id = case_data['id']
        patient_info = case_data['patient_info']

        try:
            pdf_path, md5_hash = self.get_pdf_path_from_case(case_data)
            if not pdf_path:
                return None
            file_id = self.upload_pdf_to_openai(pdf_path, md5_hash)

            thread = self.openai_client.beta.threads.create()
            self.openai_client.beta.threads.messages.create(
                thread_id=thread.id,
                role="user",
                content=f"Patient Information:\n{patient_info}\n\nPlease analyze the uploaded policy document and answer the questions.",
                attachments=[{"file_id": file_id, "tools": [{"type": "file_search"}]}]
            )

            run = self.openai_client.beta.threads.runs.create(
                thread_id=thread.id,
                assistant_id=self.assistant_id
            )
        
            while run.status in ["queued", "in_progress"]:
                time.sleep(1)
                run = self.openai_client.beta.threads.runs.retrieve(
                    thread_id=thread.id,
                    run_id=run.id
                )
            
            messages = self.openai_client.beta.threads.messages.list(thread_id=thread.id)
            result_content = messages.data[0].content[0].text.value

            result_json = self.clean_json_response(result_content)
            final_result = result_json.copy()
            final_result["md5_hash"] = md5_hash

            save_dir = f"/home/cptaswadu/new-rescue/RESCUE-n8n/eval/insurance/results/LLM_QnA/one_doc/sample/{qna_model}"
            os.makedirs(save_dir, exist_ok=True)
            filename = os.path.join(save_dir, f"{case_id}_{qna_model}_qna_result_sample.json")
            with open(filename, "w") as f:
                json.dump(final_result, f, indent=2)
            print(f"✅ QnA result saved to {filename}")
            return final_result
        
        except Exception as e:
            print(f"❗ {case_id} Error during QnA: {e}")
            return None

In [ ]:
def load_inputs(case_ex_path, ground_truth_path):
    '''Load case examples and ground-truth JSON files'''
    with open(case_ex_path, "r") as f:
        case_ex = json.load(f)

    with open(ground_truth_path, "r") as f:
        ground_truth = json.load(f)

    return case_ex, ground_truth

case_path = "/home/cptaswadu/new-rescue/RESCUE-n8n/eval/insurance/dataset/sample_qna_free_text.json"
truth_path = "/home/cptaswadu/new-rescue/RESCUE-n8n/eval/insurance/dataset/sample_ground_truth.json"

case_ex, ground_truth = load_inputs(case_path, truth_path)

In [6]:
questions_file_path = "/home/cptaswadu/new-rescue/RESCUE-n8n/eval/insurance/dataset/Insurance_Genetic_Testing_QA.json"

with open(questions_file_path, "r") as f:
    questions_data = json.load(f)

questions_list = questions_data["questions"]

In [ ]:
executor = QnAExecutorWithPDF(
    questions_list=questions_list,
    llm_model="chatgpt", 
    openai_api_key=openai_api_key
)

models_to_test = ["gpt-5-mini", "gpt-5"]
failed_cases = [] 

for model in models_to_test:
    print(f"\n🔄 Starting evaluation with model: {model}")
    for case in case_ex:
        case_id = case["id"]
        try:
            result = executor.run_qna_with_pdf(case_data=case, qna_model=model)
            if result:
                print(f"✅ {case_id} Success")
            else:
                print(f"⚠️ {case_id} Skipped (no PDF)")
        except Exception as e:
            print(f"❌ {case_id} Failed: {e}")
            failed_cases.append({"case_id": case_id, "model": model, "error": str(e)})


🔄 Starting evaluation with model: gpt-5-mini
✅ Case10917: Found PDF - United Healthcare_whole-exome-and-whole-genome-sequencing.pdf - 4fadf6b3ca9d4d08131cb31365e3aa7d


✅ OpenAI upload successed: file-Ns9qCLYBoEpWX4xoj2sieN (MD5: 4fadf6b3ca9d4d08131cb31365e3aa7d)


/tmp/ipykernel_868/3058399.py:190: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = self.openai_client.beta.threads.create()
/tmp/ipykernel_868/3058399.py:191: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  self.openai_client.beta.threads.messages.create(
/tmp/ipykernel_868/3058399.py:196: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = self.openai_client.beta.threads.runs.create(
/tmp/ipykernel_868/3058399.py:202: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = self.openai_client.beta.threads.runs.retrieve(
/tmp/ipykernel_868/3058399.py:206: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = self.openai_client.beta.threads.messages.list(



=== Case10917 LLM Response ===
```json
{
  "Q0": "WES",
  "Q1": "Not Specified",
  "Q2": "Yes",
  "Q3": "Yes",
  "Q4": "Not Specified",
  "Q5": "Yes", 
  "Q6": "Yes",
  "Q7": "81415",
  "Q8": "Yes"
}
```
✅ QnA result saved to /home/cptaswadu/new-rescue/RESCUE-n8n/eval/insurance/results/LLM_QnA/one_doc/sample/Case10917_gpt-5-mini_qna_result_sample.json
✅ Case10917 Successfully executed QnA with gpt-5-mini.
✅ Case8051: Found PDF - BCBS_FEP_204102 Whole Exome and.pdf - d5e9701c13de1dca302ad0ce45524039
✅ OpenAI upload successed: file-JJhCRjwgjoKGNbeCvrns3N (MD5: d5e9701c13de1dca302ad0ce45524039)

=== Case8051 LLM Response ===
```json
{
  "Q0": "WES",
  "Q1": "Not Specified",
  "Q2": "Not Specified",
  "Q3": "Yes",
  "Q4": "Yes",
  "Q5": "No",
  "Q6": "Yes",
  "Q7": "81415",
  "Q8": "Yes"
}
```
✅ QnA result saved to /home/cptaswadu/new-rescue/RESCUE-n8n/eval/insurance/results/LLM_QnA/one_doc/sample/Case8051_gpt-5-mini_qna_result_sample.json
✅ Case8051 Successfully executed QnA with gpt-5-m

In [ ]:
def merge_qna_jsons_to_csv(base_folder_path, model_name):
    """Merge all QnA JSON results for a specific model into a single CSV"""
    
    # Model-specific folder path
    model_folder_path = os.path.join(base_folder_path, model_name)
    
    # Check if model folder exists
    if not os.path.exists(model_folder_path):
        print(f"⚠️ Model folder not found: {model_folder_path}")
        return None
    
    all_data = []
    pattern = f"_{model_name}_qna_result_sample.json"

    # Check all files in the model folder
    for file in os.listdir(model_folder_path):
        if file.endswith(pattern):
            # Extract case_id (case123_modelname_qna_result_sample.json -> case123)
            case_id = file.replace(f"_{model_name}_qna_result_sample.json", "")
            json_path = os.path.join(model_folder_path, file)

            try:
                with open(json_path, "r", encoding='utf-8') as f:
                    result = json.load(f)
                    
                # Flatten each JSON result
                flat_result = {"case_id": case_id}
                
                for k, v in result.items():
                    if isinstance(v, list):
                        # Convert list to string separated by semicolons
                        flat_result[k] = "; ".join(map(str, v))
                    elif isinstance(v, dict):
                        # Convert dictionary to string
                        flat_result[k] = str(v)
                    else:
                        flat_result[k] = v

                all_data.append(flat_result)
                print(f"✅ Processed: {file}")
                
            except Exception as e:
                print(f"❗ Failed to parse {file}: {e}")

    # Save as CSV file
    if all_data:
        df = pd.DataFrame(all_data)
        
        # Natural sort by case_id (case1, case2, ..., case10, case11)
        def natural_sort_key(case_id):
            # Extract number from case123 -> 123
            numbers = re.findall(r'\d+', case_id)
            return int(numbers[0]) if numbers else 0
        
        df['sort_key'] = df['case_id'].apply(natural_sort_key)
        df = df.sort_values('sort_key').drop('sort_key', axis=1)
        
        # Define desired column order
        desired_columns = [
            'case_id', 'Q0', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'md5_hash'
        ]

        # Select only existing columns
        existing_columns = [col for col in desired_columns if col in df.columns]
        remaining_columns = [col for col in df.columns if col not in existing_columns]
        final_columns = existing_columns + remaining_columns
        
        # Reorder columns in desired order
        df = df[final_columns]

        # Save CSV in the model folder
        output_csv_path = os.path.join(model_folder_path, f"{model_name}_merged_results_sample.csv")
        df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

        print(f"\n✅ Successfully merged {len(all_data)} files for model: {model_name}!")
        print(f"📁 CSV saved to: {output_csv_path}")
        print(f"📊 Total records: {len(all_data)}")
        print(f"📋 Columns in order: {list(df.columns)}")
        
        return output_csv_path
    else:
        print(f"⚠️ No valid QnA result files found in: {model_folder_path}")
        return None



In [ ]:
def calculate_case_accuracies(csv_path, ground_truth):
    df = pd.read_csv(csv_path)
    questions = ["Q0", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8"]
    
    case_results = []
    for _, row in df.iterrows():
        case_id = row["case_id"]
        gt = ground_truth.get(case_id, {})
        
        correct = sum(1 for q in questions if str(row.get(q, "")).strip() == str(gt.get(q, "")).strip())
        total = 9
        accuracy = (correct / total) * 100
        
        case_results.append({
            "case_id": case_id,
            "correct": correct,
            "total": total, 
            "accuracy": accuracy
        })
    
    return case_results 


In [ ]:
def evaluate_single_model(csv_path, gold_answers, model_name):
    """Evaluate a single model's results and return case-level statistics DataFrame."""
    
    # Check existence of the CSV file
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSV file not found: {csv_path}")
    
    # Check gold_answers 
    if not isinstance(gold_answers, dict):
        raise ValueError("gold_answers must be a dictionary")
    
    df_results = pd.read_csv(csv_path)
    case_level_stats = []
    
    print(f"🔄 Evaluating {len(df_results)} cases for model: {model_name}")
    
    for _, row in df_results.iterrows():
        case_id = row['case_id']
        
        gold_result = gold_answers.get(case_id)
        if gold_result is None:
            print(f"⚠️ No gold standard found for {case_id}")
            continue
        
        predicted_result = row.to_dict()
        
        correct_count = 0
        total_count = 0
        
        # Only evaluate Q0-Q8 questions
        for qid in ["Q0", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8"]:
            if qid not in gold_result:
                continue
                
            pred_answer = predicted_result.get(qid, "")
            gold_answer = gold_result.get(qid, "")
            
            # Handle NaN cases
            if pd.isna(pred_answer):
                pred_answer = ""
            if pd.isna(gold_answer):
                gold_answer = ""

            # Handle list cases
            if isinstance(pred_answer, list):
                pred_answer = ", ".join(map(str, pred_answer))
            if isinstance(gold_answer, list):
                gold_answer = ", ".join(map(str, gold_answer))
            
            pred_answer = str(pred_answer).strip()
            gold_answer = str(gold_answer).strip()

            is_correct = pred_answer == gold_answer
            score = 1 if is_correct else 0
            
            total_count += 1
            correct_count += score
        
        accuracy = correct_count / total_count * 100 if total_count > 0 else 0
        
        case_stats = {
            "case_id": case_id,
            "model_name": model_name,
            "accuracy": accuracy,
            "correct_count": correct_count,
            "total_count": total_count
        }
        
        case_level_stats.append(case_stats)
        print(f"✅ {case_id}: {accuracy:.2f}% accuracy ({correct_count}/{total_count})")
    
    if not case_level_stats:
        print(f"⚠️ No cases were successfully evaluated for {model_name}")
        return pd.DataFrame()
        
    return pd.DataFrame(case_level_stats)

def compare_multiple_models(model_results, output_dir):
    """Compare multiple models and generate hierarchical analysis files."""
    
    # Create output directories
    os.makedirs(output_dir, exist_ok=True)
    individual_dir = os.path.join(output_dir, "individual_models")
    os.makedirs(individual_dir, exist_ok=True)

    # 1. Save individual model case-level statistics
    all_model_data = []
    model_summary_stats = []
    
    for model_name, case_df in model_results.items():
        if case_df.empty:
            continue
            
        # Save individual model case statistics
        individual_path = os.path.join(individual_dir, f"case_statistics_{model_name}.csv")
        case_df.to_csv(individual_path, index=False)
        print(f"✅ Individual model statistics saved: {individual_path}")
        
        # Collect data for comparison
        all_model_data.append(case_df)
        
        # Calculate summary statistics for each model
        accuracies = case_df["accuracy"].values
        model_stats = {
            "model_name": model_name,
            "total_cases": len(case_df),
            "mean_accuracy": accuracies.mean(),
            "std_accuracy": accuracies.std(),
            "min_accuracy": accuracies.min(),
            "max_accuracy": accuracies.max(),
            "median_accuracy": np.median(accuracies)
        }
        model_summary_stats.append(model_stats)

    # 2. Create model comparison file (model-wise summary statistics)
    if model_summary_stats:
        model_comparison_df = pd.DataFrame(model_summary_stats)
        
        # Add rankings
        model_comparison_df["accuracy_rank"] = model_comparison_df["mean_accuracy"].rank(ascending=False, method="min")
        model_comparison_df = model_comparison_df.sort_values("mean_accuracy", ascending=False)
        
        # Save model comparison file
        comparison_path = os.path.join(output_dir, "model_comparison.csv")
        model_comparison_df.to_csv(comparison_path, index=False)
        print(f"✅ Model comparison saved: {comparison_path}")
    
    # 3. Create overall analysis file (experiment wide statistics)
    if all_model_data:
        # Combine all models' data for overall experiment analysis
        combined_df = pd.concat(all_model_data, ignore_index=True)
        
        # Calculate overall experiment statistics (across all models and cases)
        all_accuracies = combined_df["accuracy"].values
        total_questions = combined_df["total_count"].sum()
        total_correct = combined_df["correct_count"].sum()
        
        overall_experiment_stats = {
            "total_models": len(model_results),
            "total_cases": len(combined_df),
            "total_questions_answered": total_questions,
            "total_correct_answers": total_correct,
            "overall_accuracy": (total_correct / total_questions * 100) if total_questions > 0 else 0,
            "mean_case_accuracy": all_accuracies.mean(),
            "std_case_accuracy": all_accuracies.std(),
            "min_case_accuracy": all_accuracies.min(),
            "max_case_accuracy": all_accuracies.max(),
            "median_case_accuracy": np.median(all_accuracies),
            "cases_with_perfect_score": len(combined_df[combined_df["accuracy"] == 100]),
            "cases_with_zero_score": len(combined_df[combined_df["accuracy"] == 0])
        }
        
        overall_df = pd.DataFrame([overall_experiment_stats])
        
        # Save overall analysis file
        overall_path = os.path.join(output_dir, "overall_analysis.csv")
        overall_df.to_csv(overall_path, index=False)
        print(f"✅ Overall analysis saved: {overall_path}")
        
        # Print summary
        print(f"\n📊 Multi-Model QnA Performance Summary:")
        print(f"Total models evaluated: {len(model_summary_stats)}")
        print(f"\n🏆 Model Rankings by Mean Accuracy:")
        for idx, row in model_comparison_df.iterrows():
            print(f"{int(row['accuracy_rank'])}. {row['model_name']}: {row['mean_accuracy']:.2f}% ± {row['std_accuracy']:.2f}%")
    
    return model_comparison_df if 'model_comparison_df' in locals() else None, overall_df if 'overall_df' in locals() else None


In [ ]:
def evaluate_csv_results(model_csvs, gold_answers, output_dir):
    """
    Evaluate multiple models and generate hierarchical analysis files.
    
    Args:
        model_csvs (dict): Dictionary of {model_name: csv_path}
        gold_answers (dict): Gold standard answers
        output_dir (str): Output directory path
    """
    
    # Evaluate each model individually
    model_results = {}
    for model_name, csv_path in model_csvs.items():
        print(f"\n🚀 Starting evaluation for model: {model_name}")
        case_df = evaluate_single_model(csv_path, gold_answers, model_name)
        if not case_df.empty:
            model_results[model_name] = case_df
    
    # Compare multiple models and generate analysis files
    if model_results:
        comparison_df, overall_df = compare_multiple_models(model_results, output_dir)
        return model_results, comparison_df, overall_df
    else:
        print("⚠️ No models were successfully evaluated")
        return None, None, None


In [ ]:
base_path = "/home/cptaswadu/new-rescue/RESCUE-n8n/eval/insurance/results/LLM_QnA/one_doc/sample"
output_dir = "/home/cptaswadu/new-rescue/RESCUE-n8n/eval/insurance/results/LLM_QnA/one_doc/sample"
model_csvs = {}

for model in models_to_test:
    print(f"\n🔄 Converting JSON to CSV for model: {model}")
    csv_path = merge_qna_jsons_to_csv(base_path, model)
    if csv_path:
        model_csvs[model] = csv_path
        print(f"✅ CSV ready for {model}: {csv_path}")
    else:
        print(f"❌ Failed to create CSV for {model}")

if model_csvs and ground_truth:
    output_dir = os.path.join(base_path, "..", "evaluation_results")
    
    model_results, comparison_df, overall_df = evaluate_csv_results(
        model_csvs=model_csvs,
        gold_answers=ground_truth,
        output_dir=output_dir
    )
    
    print(f"\n🎉 Analysis complete! Results saved to: {output_dir}")
else:
    print("❌ Could not run evaluation - missing CSV files or gold answers")

if failed_cases:
    print(f"\n⚠️ Failed Cases Summary ({len(failed_cases)} total):")
    for fail in failed_cases:
        print(f"  - {fail['case_id']} ({fail['qna_model']}): {fail['error']}")



🔄 Processing model: gpt-4o
⚠️ No valid QnA result files found in: /home/cptaswadu/new-rescue/RESCUE-n8n/eval/insurance/results/LLM_QnA/one_doc/sample
⚠️ No CSV file generated for gpt-4o

🔄 Processing model: gpt-5-nano
⚠️ No valid QnA result files found in: /home/cptaswadu/new-rescue/RESCUE-n8n/eval/insurance/results/LLM_QnA/one_doc/sample
⚠️ No CSV file generated for gpt-5-nano

🔄 Processing model: gpt-5-mini
✅ Processed: Case19321_gpt-5-mini_qna_result_sample.json
✅ Processed: Case8051_gpt-5-mini_qna_result_sample.json
✅ Processed: Case18257_gpt-5-mini_qna_result_sample.json
✅ Processed: Case10363_gpt-5-mini_qna_result_sample.json
✅ Processed: Case4512_gpt-5-mini_qna_result_sample.json
✅ Processed: Case7376_gpt-5-mini_qna_result_sample.json
✅ Processed: Case10917_gpt-5-mini_qna_result_sample.json
✅ Processed: Case11124_gpt-5-mini_qna_result_sample.json
✅ Processed: Case9349_gpt-5-mini_qna_result_sample.json
✅ Processed: Case10451_gpt-5-mini_qna_result_sample.json

✅ Successfully merg

In [11]:
def analyze_question_accuracy(folder_path, ground_truth, models_to_test, output_dir):
    """Analyze question-wise accuracy - overall and by model"""
    
    # Store results
    all_results = {}
    question_columns = ['Q0', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8']
    
    # Collect data for each model
    for model in models_to_test:
        csv_path = os.path.join(folder_path, f"{model}_merged_results_sample.csv")
        
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            model_accuracy = {}
            
            # Calculate accuracy for each question
            for question in question_columns:
                if question in df.columns:
                    correct_count = 0
                    total_count = 0
                    
                    for _, row in df.iterrows():
                        case_id = row['case_id']
                        if case_id in ground_truth:
                            predicted = str(row[question]).strip()
                            actual = str(ground_truth[case_id][question]).strip()
                            
                            total_count += 1
                            if predicted == actual:
                                correct_count += 1
                    
                    accuracy = correct_count / total_count if total_count > 0 else 0
                    model_accuracy[question] = {
                        'accuracy': accuracy,
                        'correct': correct_count,
                        'total': total_count
                    }
            
            all_results[model] = model_accuracy
            print(f"✅ Processed {model}: {len(df)} cases")
    
    # 1. Overall analysis across all models
    print("\n" + "="*60)
    print("📊 OVERALL QUESTION ACCURACY (All Models Combined)")
    print("="*60)
    
    overall_stats = {}
    for question in question_columns:
        total_correct = sum([all_results[model][question]['correct'] 
                            for model in all_results.keys() 
                            if question in all_results[model]])
        total_cases = sum([all_results[model][question]['total'] 
                          for model in all_results.keys() 
                          if question in all_results[model]])
        
        overall_accuracy = total_correct / total_cases if total_cases > 0 else 0
        overall_stats[question] = {
            'accuracy': overall_accuracy,
            'correct': total_correct,
            'total': total_cases
        }
        
        print(f"{question}: {overall_accuracy:.3f} ({total_correct}/{total_cases})")
    
    # 2. Model-wise analysis
    print("\n" + "="*60)
    print("📊 MODEL-WISE QUESTION ACCURACY")
    print("="*60)
    
    # Create results table
    results_df = pd.DataFrame()
    for model in models_to_test:
        if model in all_results:
            model_row = {'Model': model}
            for question in question_columns:
                if question in all_results[model]:
                    accuracy = all_results[model][question]['accuracy']
                    model_row[question] = accuracy
                else:
                    model_row[question] = 0
            results_df = pd.concat([results_df, pd.DataFrame([model_row])], ignore_index=True)
    
    print(results_df.round(3).to_string(index=False))
    
    # 3. Question difficulty ranking (easiest to hardest)
    print("\n" + "="*60)
    print("📊 QUESTION DIFFICULTY RANKING")
    print("="*60)
    
    difficulty_ranking = sorted(overall_stats.items(), 
                               key=lambda x: x[1]['accuracy'], reverse=True)
    
    difficulty_df = pd.DataFrame([
        {
            'Question': q,
            'Overall_Accuracy': stats['accuracy'],
            'Correct_Total': f"{stats['correct']}/{stats['total']}",
            'Difficulty_Level': 'Easy' if stats['accuracy'] > 0.8 else 'Medium' if stats['accuracy'] > 0.5 else 'Hard'
        }
        for q, stats in difficulty_ranking
    ])
    
    print(difficulty_df.to_string(index=False))
    
    # 4. Save results to files
    timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
    
    # Save overall results
    overall_df = pd.DataFrame([
        {
            'Question': q,
            'Accuracy': stats['accuracy'],
            'Correct': stats['correct'],
            'Total': stats['total']
        }
        for q, stats in overall_stats.items()
    ])
    
    overall_output_path = os.path.join(output_dir, f"overall_question_accuracy_{timestamp}.csv")
    overall_df.to_csv(overall_output_path, index=False)
    print(f"\n✅ Overall results saved: {overall_output_path}")
    
    # Save model-wise results
    model_output_path = os.path.join(output_dir, f"model_wise_question_accuracy_{timestamp}.csv")
    results_df.to_csv(model_output_path, index=False)
    print(f"✅ Model-wise results saved: {model_output_path}")
    
    # Save difficulty ranking
    difficulty_output_path = os.path.join(output_dir, f"question_difficulty_ranking_{timestamp}.csv")
    difficulty_df.to_csv(difficulty_output_path, index=False)
    print(f"✅ Difficulty ranking saved: {difficulty_output_path}")
    
    # Save detailed results (JSON format for further analysis)
    detailed_output_path = os.path.join(output_dir, f"detailed_question_analysis_{timestamp}.json")
    with open(detailed_output_path, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)
    print(f"✅ Detailed results saved: {detailed_output_path}")
    
    return overall_stats, results_df, difficulty_df

# Run analysis
overall_stats, model_results, difficulty_ranking = analyze_question_accuracy(
    folder_path=folder_path,
    ground_truth=ground_truth,
    models_to_test=models_to_test,
    output_dir=output_dir
)


✅ Processed gpt-5-mini: 10 cases

📊 OVERALL QUESTION ACCURACY (All Models Combined)
Q0: 1.000 (10/10)
Q1: 0.300 (3/10)
Q2: 0.800 (8/10)
Q3: 0.900 (9/10)
Q4: 0.600 (6/10)
Q5: 0.800 (8/10)
Q6: 0.400 (4/10)
Q7: 0.800 (8/10)
Q8: 0.900 (9/10)

📊 MODEL-WISE QUESTION ACCURACY
     Model  Q0  Q1  Q2  Q3  Q4  Q5  Q6  Q7  Q8
gpt-5-mini 1.0 0.3 0.8 0.9 0.6 0.8 0.4 0.8 0.9

📊 QUESTION DIFFICULTY RANKING
Question  Overall_Accuracy Correct_Total Difficulty_Level
      Q0               1.0         10/10             Easy
      Q3               0.9          9/10             Easy
      Q8               0.9          9/10             Easy
      Q2               0.8          8/10           Medium
      Q5               0.8          8/10           Medium
      Q7               0.8          8/10           Medium
      Q4               0.6          6/10           Medium
      Q6               0.4          4/10             Hard
      Q1               0.3          3/10             Hard

✅ Overall results saved:

In [12]:
def create_evaluation_figures(case_df, overall_df, output_dir):
    """Create various evaluation figures based on the case_df and overall_df data."""
    
    # Font
    plt.rcParams['font.family'] = 'DejaVu Sans'
    
    # 1. Bar chart of accuracy by case
    plt.figure(figsize=(12, 6))
    case_df_sorted = case_df.sort_values('case_id', key=lambda x: x.str.extract('(\d+)').astype(int)[0])
    
    colors = ['#ff7f7f' if acc < 70 else '#ffb347' if acc < 80 else '#90EE90' for acc in case_df_sorted['accuracy']]
    
    bars = plt.bar(range(len(case_df_sorted)), case_df_sorted['accuracy'], 
                   color=colors, alpha=0.8, edgecolor='black', linewidth=0.5)
    
    # Add value labels on top of each bar
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{height:.1f}%', ha='center', va='bottom', fontsize=9)
    
    plt.xlabel('Case ID')
    plt.ylabel('Accuracy (%)')
    plt.title('Accuracy by Case')
    plt.xticks(range(len(case_df_sorted)), case_df_sorted['case_id'], rotation=45)
    plt.ylim(0, 105)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    
    bar_path = os.path.join(output_dir, "accuracy_by_case.png")
    plt.savefig(bar_path, dpi=300, bbox_inches='tight')
    print(f"📊 Case accuracy bar chart saved to: {bar_path}")
    plt.show()
    plt.close()
    
    # 2. Histogram of accuracy distribution
    plt.figure(figsize=(10, 6))
    plt.hist(case_df['accuracy'], bins=15, alpha=0.7, color='skyblue', 
             edgecolor='black', linewidth=0.5)
    
    mean_acc = case_df['accuracy'].mean()
    plt.axvline(mean_acc, color='red', linestyle='--', linewidth=2, 
                label=f'Mean: {mean_acc:.1f}%')
    
    plt.xlabel('Accuracy (%)')
    plt.ylabel('Number of Cases')
    plt.title(f'Distribution of Case Accuracy\n(Mean: {mean_acc:.1f}%, Std: {case_df["accuracy"].std():.1f}%)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    hist_path = os.path.join(output_dir, "accuracy_distribution.png")
    plt.savefig(hist_path, dpi=300, bbox_inches='tight')
    print(f"📊 Accuracy distribution saved to: {hist_path}")
    plt.show()
    plt.close()
    
    # 3. Bar charts for correct and total question counts by case
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 3-1. Bar chart for correct answers
    bars1 = ax1.bar(range(len(case_df_sorted)), case_df_sorted['correct_count'], color='lightgreen', alpha=0.8, edgecolor='black', linewidth=0.5)
    
    for i, bar in enumerate(bars1):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{int(height)}', ha='center', va='bottom', fontsize=9)
    
    ax1.set_xlabel('Case ID')
    ax1.set_ylabel('Correct Count')
    ax1.set_title('Number of Correct Answers by Case')
    ax1.set_xticks(range(len(case_df_sorted)))
    ax1.set_xticklabels(case_df_sorted['case_id'], rotation=45)
    ax1.grid(True, alpha=0.3, axis='y')
    
    # 3-2. Bar chart for total questions
    bars2 = ax2.bar(range(len(case_df_sorted)), case_df_sorted['total_count'], 
                    color='lightcoral', alpha=0.8, edgecolor='black', linewidth=0.5)
    
    for i, bar in enumerate(bars2):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{int(height)}', ha='center', va='bottom', fontsize=9)
    
    ax2.set_xlabel('Case ID')
    ax2.set_ylabel('Total Questions')
    ax2.set_title('Total Number of Questions by Case')
    ax2.set_xticks(range(len(case_df_sorted)))
    ax2.set_xticklabels(case_df_sorted['case_id'], rotation=45)
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    count_path = os.path.join(output_dir, "question_counts_by_case.png")
    plt.savefig(count_path, dpi=300, bbox_inches='tight')
    print(f"📊 Question counts chart saved to: {count_path}")
    plt.show()
    plt.close()
    
    # 4. Pie chart of performance categories
    plt.figure(figsize=(8, 6))
    
    high_perf = len(case_df[case_df['accuracy'] >= 85])
    mid_perf = len(case_df[(case_df['accuracy'] >= 70) & (case_df['accuracy'] < 85)])
    low_perf = len(case_df[case_df['accuracy'] < 70])
    
    labels = ['High (≥85%)', 'Medium (70-84%)', 'Low (<70%)']
    sizes = [high_perf, mid_perf, low_perf]
    colors = ['#90EE90', '#ffb347', '#ff7f7f']
    
    non_zero = [(label, size, color) for label, size, color in zip(labels, sizes, colors) if size > 0]
    if non_zero:
        labels, sizes, colors = zip(*non_zero)
        
        plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', 
                startangle=90, explode=[0.05]*len(sizes))
        plt.title('Performance Distribution by Category')
        plt.axis('equal')
        
        pie_path = os.path.join(output_dir, "performance_categories.png")
        plt.savefig(pie_path, dpi=300, bbox_inches='tight')
        print(f"📊 Performance categories pie chart saved to: {pie_path}")
        plt.show()
        plt.close()
    
    # 5. Overall evaluation summary
    plt.figure(figsize=(10, 6))
    plt.axis('off')
    
    stats_text = f"""
    📊 Overall Evaluation Statistics
    
    Total Cases Evaluated: {overall_df.iloc[0]['total_cases']}
    
    Accuracy Statistics:
    • Mean: {overall_df.iloc[0]['mean_accuracy']:.2f}%
    • Standard Deviation: {overall_df.iloc[0]['std_accuracy']:.2f}%
    • Minimum: {overall_df.iloc[0]['min_accuracy']:.2f}%
    • Maximum: {overall_df.iloc[0]['max_accuracy']:.2f}%
    • Median: {overall_df.iloc[0]['median_accuracy']:.2f}%
    • 25th Percentile: {overall_df.iloc[0]['q25_accuracy']:.2f}%
    • 75th Percentile: {overall_df.iloc[0]['q75_accuracy']:.2f}%
    
    Performance Categories:
    • High Performance (≥85%): {high_perf} cases
    • Medium Performance (70-84%): {mid_perf} cases
    • Low Performance (<70%): {low_perf} cases
    """
    
    plt.text(0.1, 0.5, stats_text, fontsize=12, verticalalignment='center',
             bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))
    
    plt.title('Evaluation Summary Statistics', fontsize=16, fontweight='bold', pad=20)
    
    summary_path = os.path.join(output_dir, "evaluation_summary.png")
    plt.savefig(summary_path, dpi=300, bbox_inches='tight')
    print(f"📊 Evaluation summary saved to: {summary_path}")
    plt.show()
    plt.close() 


<>:9: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_868/2262711693.py:9: SyntaxWarning: invalid escape sequence '\d'
  case_df_sorted = case_df.sort_values('case_id', key=lambda x: x.str.extract('(\d+)').astype(int)[0])


In [13]:
def evaluate_csv_results_with_figures(csv_path, gold_answers, output_dir):
    """Evaluate CSV results and generate visualization figures."""
    
    case_df, overall_df = evaluate_csv_results(csv_path, gold_answers, output_dir)
    
    if case_df is not None and overall_df is not None:
        print(f"\n🎨 Generating visualization figures...")
        create_evaluation_figures(case_df, overall_df, output_dir)
        print(f"✅ All figures saved to: {output_dir}")
    
    return case_df, overall_df


# case_df, overall_df = evaluate_csv_results_with_figures(csv_path, ground_truth, output_dir)